# Vaccinations Administered by County

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json
from scipy import stats
from api_keys import geoapify_key
from api_keys import google_api_key

In [2]:
# Load vaccine data into DataFrame
vaccine_df = pd.read_csv("../Keyana/COVID-19_Vaccinations_in_the_United_States_County.csv")
vaccine_df.head()

,Recip_County,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_65Plus,Series_Complete_5Plus,Series_Complete_12Plus,Series_Complete_18Plus,Series_Complete_65Plus
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0
4,Saginaw County,NaN,NaN,NaN,NaN,103054.0,101354.0,95769.0,32377.0


In [3]:
# Rename columns
vaccine_df = vaccine_df.rename(columns = {'Recip_County': 'County',
        'Administered_Dose1_Recip_5Plus': 'Administered Dose 1 (5 Plus)',
        'Administered_Dose1_Recip_12Plus': 'Administered Dose 1 (12 Plus)',
        'Administered_Dose1_Recip_18Plus': 'Administered Dose 1 (18 Plus)',
        'Administered_Dose1_Recip_65Plus': 'Administered Dose 1 (65 Plus)',
        'Series_Complete_5Plus': 'Administered Series (5 Plus)',
        'Series_Complete_12Plus': 'Administered Series (12 Plus)',
        'Series_Complete_18Plus': 'Administered Series (18 Plus)',
        'Series_Complete_65Plus': 'Administered Series (65 Plus)'})

vaccine_df

,County,Administered Dose 1 (5 Plus),Administered Dose 1 (12 Plus),Administered Dose 1 (18 Plus),Administered Dose 1 (65 Plus),Administered Series (5 Plus),Administered Series (12 Plus),Administered Series (18 Plus),Administered Series (65 Plus)
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0
4,Saginaw County,NaN,NaN,NaN,NaN,103054.0,101354.0,95769.0,32377.0
...,...,...,...,...,...,...,...,...,...
1962776,Adams County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1962777,Harrison County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1962778,Matagorda County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1962779,Pleasants County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0


In [4]:
# Add column for total number of vaccines 
vaccine_df['Total Vaccines Administered'] = vaccine_df['Administered Dose 1 (5 Plus)'] + vaccine_df['Administered Dose 1 (12 Plus)'] + vaccine_df['Administered Dose 1 (18 Plus)'] + vaccine_df['Administered Dose 1 (65 Plus)'] + vaccine_df['Administered Series (5 Plus)'] + vaccine_df['Administered Series (12 Plus)'] + vaccine_df['Administered Series (18 Plus)'] + vaccine_df['Administered Series (65 Plus)']
vaccine_df


,County,Administered Dose 1 (5 Plus),Administered Dose 1 (12 Plus),Administered Dose 1 (18 Plus),Administered Dose 1 (65 Plus),Administered Series (5 Plus),Administered Series (12 Plus),Administered Series (18 Plus),Administered Series (65 Plus),Total Vaccines Administered
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0,71639.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0,20278.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0,7872069.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0,70519.0
4,Saginaw County,NaN,NaN,NaN,NaN,103054.0,101354.0,95769.0,32377.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1962776,Adams County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
1962777,Harrison County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
1962778,Matagorda County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
1962779,Pleasants County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN


In [5]:
# Check for duplicates
duplicates = vaccine_df['County'].duplicated
print(duplicates)

<bound method Series.duplicated of 0           Washburn County
1             Taylor County
2             Nassau County
3           Lampasas County
4            Saginaw County
                 ...       
1962776        Adams County
1962777     Harrison County
1962778    Matagorda County
1962779    Pleasants County
1962780     Houghton County
Name: County, Length: 1962781, dtype: object>


In [6]:
# Remove duplicates
vaccine_df = vaccine_df.drop_duplicates(subset=['County'], keep='first')
vaccine_df

,County,Administered Dose 1 (5 Plus),Administered Dose 1 (12 Plus),Administered Dose 1 (18 Plus),Administered Dose 1 (65 Plus),Administered Series (5 Plus),Administered Series (12 Plus),Administered Series (18 Plus),Administered Series (65 Plus),Total Vaccines Administered
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0,71639.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0,20278.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0,7872069.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0,70519.0
4,Saginaw County,NaN,NaN,NaN,NaN,103054.0,101354.0,95769.0,32377.0,NaN
...,...,...,...,...,...,...,...,...,...,...
3276,Swift County,5893.0,5636.0,5277.0,1955.0,5537.0,5299.0,4964.0,1867.0,36428.0
3277,Stokes County,29136.0,28648.0,27464.0,10293.0,23030.0,22666.0,21725.0,8347.0,171309.0
3278,Ozark County,3760.0,3723.0,3635.0,1706.0,3055.0,3033.0,2970.0,1402.0,23284.0
3281,Churchill County,16164.0,15826.0,14980.0,4504.0,13979.0,13710.0,12974.0,3909.0,96046.0


In [7]:
# Drop NaN
vaccine_df.dropna()

,County,Administered Dose 1 (5 Plus),Administered Dose 1 (12 Plus),Administered Dose 1 (18 Plus),Administered Dose 1 (65 Plus),Administered Series (5 Plus),Administered Series (12 Plus),Administered Series (18 Plus),Administered Series (65 Plus),Total Vaccines Administered
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0,71639.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0,20278.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0,7872069.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0,70519.0
5,Echols County,1572.0,1537.0,1458.0,366.0,1273.0,1245.0,1182.0,319.0,8952.0
...,...,...,...,...,...,...,...,...,...,...
3276,Swift County,5893.0,5636.0,5277.0,1955.0,5537.0,5299.0,4964.0,1867.0,36428.0
3277,Stokes County,29136.0,28648.0,27464.0,10293.0,23030.0,22666.0,21725.0,8347.0,171309.0
3278,Ozark County,3760.0,3723.0,3635.0,1706.0,3055.0,3033.0,2970.0,1402.0,23284.0
3281,Churchill County,16164.0,15826.0,14980.0,4504.0,13979.0,13710.0,12974.0,3909.0,96046.0


In [8]:
# API and response
county = vaccine_df['County']
url_county = f"https://maps.googleapis.com/maps/api/geocode/json?address={county}&key={google_api_key}"
response_county = requests.get(url_county).json()
print(json.dumps(response_county, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Lampasas County",
                    "short_name": "Lampasas County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Texas",
                    "short_name": "TX",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "United States",
                    "short_name": "US",
                    "types": [
                        "country",
                        "political"
                    ]
                }
            ],
            "formatted_address": "Lampasas County, TX, USA",
            "geometry": {
                "bounds": {
      

In [13]:
# List to hold lat, lon and county data
county_list = []
lat = []
lon = []


# Find latitude and longitude for each county
countys = vaccine_df['County']
for county in countys:
    try:
        response = requests.get(url_county + county).json()
        county_list.append(response['results'][0]['address_components'][0]['long_name'])
        lat.append(response['results'][0]['geometry']['location']['lat'])
        lon.append(response['results'][0]['geometry']['location']['lng'])
    except (KeyError, IndexError):
        pass
print(lat)
print(lon)
print(county_list)

[]
[]
[]


In [ ]:
# Create DataFrame for mapping
county_dict = {
    "County": county_list,
    "Lat": lat,
    "Lon": lon,
    "Total Vaccines Administered": vaccine_df['Total Vaccines Administered']
}
county_vaccine_df = pd.DataFrame(county_dict)
county_vaccine_df

In [ ]:
# Configure map
map_plot_1 = county_vaccine_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Total Vaccines Administrated",
    scale = 0.5, 
    color = "County"
)

# Display the map plot
map_plot_1

In [ ]:
# Statistical summary for total 1 dose  (need to change)
#Alter once totals for vaccinations of dose 1 and series calculated
mean_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).sem()

mean_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).sem()

mean_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).sem()

mean_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).sem()

summary_dose1_df = pd.DataFrame({"Mean Dose 1 Vaccine Distribution (Age 5+)": mean_dose1_5,
                           "Mean Dose 1 Vaccine Distribution (Age 12+)": mean_dose1_12,
                           "Mean Dose 1 Vaccine Distribution (Age 18+)": mean_dose1_18,
                           "Mean Dose 1 Vaccine Distribution (Age 65+)": mean_dose1_65,
                           "Median Dose 1 Vaccine Distribution (Age 5+)": median_dose1_5,
                           "Median Dose 1 Vaccine Distribution (Age 12+)": median_dose1_12,
                           "Median Dose 1 Vaccine Distribution (Age 18+)": median_dose1_18,
                           "Median Dose 1 Vaccine Distribution (Age 65+)": median_dose1_65,
                           "Dose 1 Vaccine Distribution Variance (Age 5+)": variance_dose1_5,
                           "Dose 1 Vaccine Distribution Variance (Age 12+)": variance_dose1_12,
                           "Dose 1 Vaccine Distribution Variance (Age 18+)": variance_dose1_18,
                           "Dose 1 Vaccine Distribution Variance (Age 65+)": variance_dose1_65,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 5+)": std_dev_dose1_5,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 12+)": std_dev_dose1_12,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 18+)": std_dev_dose1_18,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 65+)": std_dev_dose1_65,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 5+)": sem_dose1_5,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 12+)": sem_dose1_12,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 18+)": sem_dose1_18,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 65+)": sem_dose1_65})
summary_dose1_df

In [ ]:
# Statistical summary for series
mean_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).mean()
median_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).median()
variance_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).std()
sem_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).sem()

mean_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).mean()
median_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).median()
variance_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).std()
sem_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).sem()

mean_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).mean()
median_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).median()
variance_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).std()
sem_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).sem()

mean_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).mean()
median_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).median()
variance_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).std()
sem_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).sem()

summary_series_df = pd.DataFrame({"Mean Series Vaccine Distribution (Age 5+)": mean_series_5,
                                  "Mean Series Vaccine Distribution (Age 12+)": mean_series_12,
                                  "Mean Series Vaccine Distribution (Age 18+)": mean_series_18,
                                  "Mean Series Vaccine Distribution (Age 65+)": mean_series_65,
                                  "Median Series Vaccine Distribution (Age 5+)": median_series_5,
                                  "Median Series Vaccine Distribution (Age 12+)": median_series_12,
                                  "Median Series Vaccine Distribution (Age 18+)": median_series_18,
                                  "Median Series Vaccine Distribution (Age 65+)": median_series_65,
                                  "Series Vaccine Distribution Variance (Age 5+)": variance_series_5,
                                  "Series Vaccine Distribution Variance (Age 12+)": variance_series_12,
                                  "Series Vaccine Distribution Variance (Age 18+)": variance_series_18,
                                  "Series Vaccine Distribution Variance (Age 65+)": variance_series_65,
                                  "Series Vaccine Distribution Std. Dev (Age 5+)": std_dev_series_5,
                                  "Series Vaccine Distribution Std. Dev (Age 12+)": std_dev_series_12,
                                  "Series Vaccine Distribution Std. Dev (Age 18+)": std_dev_series_18,
                                  "Series Vaccine Distribution Std. Dev (Age 65+)": std_dev_series_65,
                                  "Series Vaccine Distribution Std. Err (Age 5+)": sem_series_5,
                                  "Series Vaccine Distribution Std. Err (Age 12+)": sem_series_12,
                                  "Series Vaccine Distribution Std. Err (Age 18+)": sem_series_18,
                                  "Series Vaccine Distribution Std. Err (Age 65+)": sem_series_65})
summary_series_df